<a href="https://colab.research.google.com/github/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch/UsingCppModules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using CPP modules

Writing parts of a Deep Learning pipeline in C++ unlocks many oportunities for optimizing speed and resource consumption.

PyTorch provides a very nice and easy to use interface for writing a module in C++, compiling it and using it in Python.

In this lesson we learn how to implement the Macro F1 Score in C++ and use it in Python. However, the real performance of C++ is seen when moving even more complex Python based processing to C++.

## Setup

Here we get the `optimized_f1_score` module.

In [1]:
import shutil
import os
import subprocess

if not os.path.isdir("optimized_f1_score"):
    subprocess.run(["git", "clone", "https://www.github.com/Tensor-Reloaded/AI-Learning-Hub"], check=True)
    shutil.copytree("AI-Learning-Hub/resources/advanced_pytorch/optimized_f1_score", "optimized_f1_score")
    shutil.rmtree("AI-Learning-Hub", ignore_errors=True)


Let's see the content of each file. First we have the Macro F1 Score implemented in Python.

In [2]:
!pygmentize -g optimized_f1_score/f1_macro_py.py

import torch


def f1_score(x: torch.BoolTensor, y: torch.BoolTensor) -> float:
    x_sum = x.sum()
    y_sum = y.sum()
    if x_sum == 0 or y_sum == 0:
        if x_sum == 0 and y_sum == 0:
            return 1.0
        return 0.0
    return 2 * (x & y).sum() / (x_sum + y_sum)


def f1_macro(x: torch.Tensor, y: torch.Tensor, classes: int) -> float:
    result = 0.0
    for c in range(classes):
        result += f1_score(x == c, y == c)
    return result / classes


Next we have the same function implemented in C++.

In [3]:
!pygmentize -g optimized_f1_score/f1_macro_cpp.cpp

#include <torch/extension.h>

double f1_score(torch::Tensor x, torch::Tensor y) {
    auto x_sum = x.sum().item<double>();
    auto y_sum = y.sum().item<double>();

    if (x_sum == 0.0 || y_sum == 0.0) {
        if (x_sum == 0.0 && y_sum == 0.0)
            return 1.0;
        return 0.0;
    }

    auto intersection = (x & y).sum().item<double>();
    return 2.0 * intersection / (x_sum + y_sum);
}

double f1_macro(torch::Tensor x, torch::Tensor y, int classes) {
    // You can disable the checks if you own this function. Never do this if you can't control the input.
    TORCH_CHECK(x.sizes() == y.sizes(), "Input tensors must have the same shape");
    TORCH_CHECK(x.device() == y.device(), "Input tensors must be on the same device");

    double result = 0.0;
    for (int c = 0; c < classes; ++c) {
        auto x_mask = (x == c);
        auto y_mask = (y == c);
        result += f1_score(x_mask, y_mask);
    }
    return result / classes;
}

// Bind functions to Python
PYBIND11_MODULE

If you can read the code above, you can see that the C++ implementation of `f1_macro` and `f1_score` is very similar to the Python implementation.

Whenever we write a C++ module, we need to include the torch extensions (`#include <torch/extension.h>`) and we need to use the `PYBIND11_MODULE` macro to bind the functions implemented in C++ to Python, allowing them to be called just like regular Python functions.

To build the C++ code, we need a `setup.py` file just like we would build any Python library from source.

In [4]:
!pygmentize -g optimized_f1_score/setup.py

import os

from setuptools import setup
from torch.utils.cpp_extension import CppExtension, BuildExtension


def get_extra_compile_args():
    if os.name == "nt":
        return {
            "msvc": ["/std:c++20", "/O2", "/DNDEBUG",
                     # "/arch:AVX2"
                     ],
        }
    else:
        return {
            "cxx": ["-std=c++20", "-O3", "-DNDEBUG",
                    # "-mavx2"
                    ],
        }


if __name__ == "__main__":
    setup(
        name="optimized_f1_score",
        ext_modules=[
            CppExtension(
                "f1_macro_cpp",
                ["f1_macro_cpp.cpp"],
                extra_compile_args=get_extra_compile_args(),
                define_macros=[
                    ("ARCH_DEFAULT", None)
                ],
            ),
        ],
        cmdclass={"build_ext": BuildExtension},
    )


Above you have the `setup.py` file. In this file you can set compile options that can speed the C++ code using vectorization depending on your architecture.
For example, I can use `/arch:AVX2` or `-mavx2` to tell the compiler that it can use 256-bit SIMD instructions (see the commented out code).

On most non-Mac modern hardware, using AVX2 is a great way of speeding up your tensor computations using vectorized instructions.
Depending on your hardware, you can also use `AVX`, `SSE`, `SSE2`, and so on.

You can also enable fused multiply add or fast math (if you know what you are doing).

In [5]:
!pygmentize -g optimized_f1_score/build.py

import os
import subprocess
import filelock


def build():
    p_dir = os.path.dirname(__file__)
    lock_path = os.path.join(p_dir, ".lock")

    try:
        from optimized_f1_score.f1_macro_cpp import f1_macro
    except ImportError:
        with filelock.FileLock(lock_path):
            print("Building Optimized F1 Score")
            try_build(p_dir)


def try_build(p_dir):
    try:
        setup_cmd = f'python setup.py build_ext --inplace'
        if os.name == "nt":
            setup_cmd = prepare_windows_build() + setup_cmd
        subprocess.run(setup_cmd, shell=True, cwd=p_dir)
    except Exception as e:
        print(f"Error during build. Please check logs. See {e}")
    print("Done building")


def prepare_windows_build():
    activate_msvc = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\VC\\Auxiliary\\Build\\vcvars64.bat"
    if "ACTIVATE_MSVC" in os.environ:
        activate_msvc = os.environ["ACTIVATE_MSVC"]
    if not os.path.isfile(activate_msvc):
     

The build file above allows us to dynamically build the C++ module as a Python library. We just need to have a compatible C++ compiler vizible in the path (which means that we have to activate MSVC on Windows).

In [6]:
!pygmentize -g optimized_f1_score/__init__.py

from optimized_f1_score.build import build



try:
    from optimized_f1_score.f1_macro_cpp import f1_macro
except ImportError as e:
    try:
        build()
        from optimized_f1_score.f1_macro_cpp import f1_macro
    except ImportError as e:
        print("Building Optimized F1 Score failed. Using Fallback")
        from optimized_f1_score.f1_macro_py import f1_macro


This is how we implement dynamic building and importing of a C++ module, with a fallback that allows us to use the Python implementation if compilation failed.

In [7]:
import torch
from optimized_f1_score import f1_macro_cpp, f1_macro_py

Building Optimized F1 Score
Done building


In [8]:
num_classes = 100
size = 10000

In [9]:
torch.random.manual_seed(3)
x = torch.randint(0, num_classes, (size,))
y = torch.randint(0, num_classes, (size,))
x_cuda = x.cuda()
y_cuda = y.cuda()


In [10]:
%%timeit
f1_macro_cpp.f1_macro(x, y, num_classes)

4.2 ms ± 82.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit
f1_macro_py.f1_macro(x, y, num_classes)

8.87 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
f1_macro_cpp.f1_macro(x_cuda, y_cuda, num_classes)

14.3 ms ± 841 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
f1_macro_py.f1_macro(x_cuda, y_cuda, num_classes)

18.5 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can see that the C++ module is faster than the Python implementation.
And hey, we can also make our C++ code faster by releasing the GIL lock and using threads!

Writing data processing pipelines in C++ can make your code 10x-100x faster or even more (if you know what are you doing)!

The benefits of using C++ are mostly seen on CPU tensors. To speed up GPU computation even more, you usually need to implement custom CUDA kernels, which is not covered in this tutorial.

---

### Additional resources:
* Check the cppdocs, torch has a really nice library interfacing with C++: https://docs.pytorch.org/cppdocs/
* Check the official PyTorch tutorial for custom C++ operators: https://docs.pytorch.org/tutorials/advanced/cpp_custom_ops.html
* This is the official documentation for CppExtension: https://docs.pytorch.org/docs/stable/cpp_extension.html
* This tutorial shows how to optimize the data processing pipeline with custom PyTorch operators: https://medium.com/data-science/how-to-optimize-your-dl-data-input-pipeline-with-a-custom-pytorch-operator-7f8ea2da5206

---

| All     | [advanced_pytorch/](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch) |
|---------|-- |
| Current | [Using Cpp Modules](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch/UsingCppModules.ipynb) |